https://www.kaggle.com/c/integer-sequence-learning

# Libraries

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# Data

In [2]:
data = pd.read_csv('Sequences.csv')

In [3]:
data.head()

,Sequence
0,"1,3,13,87,1053,28576,2141733,508147108,4021352..."
1,"1,2,1,5,5,1,11,16,7,1,23,44,30,9,1,47,112,104,..."
2,"1,2,4,5,8,10,16,20,32,40,64,80,128,160,256,320..."
3,"1,8,25,83,274,2275,132224,1060067,3312425,1099..."
4,"1,111,12211,1343211,147753211,16252853211,1787..."


In [4]:
len(data)

227690

In [5]:
data['Sequence'] = list(map(lambda x: list(map(int, x.split(','))), data['Sequence']))

In [6]:
data['Sequence'][0]

[1,
 3,
 13,
 87,
 1053,
 28576,
 2141733,
 508147108,
 402135275365,
 1073376057490373,
 9700385489355970183,
 298434346895322960005291,
 31479360095907908092817694945,
 11474377948948020660089085281068730]

In [7]:
data['Solved'] = ''

In [8]:
data

,Sequence,Solved
0,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",
1,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9...",
2,"[1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12...",
3,"[1, 8, 25, 83, 274, 2275, 132224, 1060067, 331...",
4,"[1, 111, 12211, 1343211, 147753211, 1625285321...",
...,...,...
227685,"[0, 2, 6, 20, 72, 274, 1086, 4438, 18570, 7917...",
227686,"[1, 15, 208, 2389, 24845, 243697, 2348885, 228...",
227687,"[0, 0, 2, 3, 5, 7, 11, 13, 24, 30, 36, 46, 50,...",
227688,"[1, 3, 5, 9, 15, 25, 27, 45, 75, 81, 125, 135,...",


# Sequences

## Recurrence Relations

In [9]:
def checkRecurrence(seq, order= 2, minlength = 7):

    if len(seq)< max((2*order+1), minlength):
        return None
    
    # Set up the system of equations 
    A,b = [], []
    for i in range(order):
        A.append(seq[i:i+order])
        b.append(seq[i+order])
    A,b =np.array(A), np.array(b)
    try: 
        if np.linalg.det(A)==0:
            return None
    except TypeError:
        return None
   
    #  Solve for the coefficients (c0, c1, c2, ...)
    coeffs = np.linalg.inv(A).dot(b)  
    
    #  Check if the next terms satisfy recurrence relation
    for i in range(2*order, len(seq)):
        predict = np.sum(coeffs*np.array(seq[i-order:i]))
        if abs(predict-seq[i])>10**(-2):
            return None
    
    return list(coeffs)

In [10]:
def predictNextTerm(seq, coeffs):
    order = len(coeffs)
    predict = np.sum(coeffs*np.array(seq[-order:]))
    return int(round(predict))

### 2nd Order Sequences

In [11]:
order2Seq = 0
for i, seq in tqdm(enumerate(data['Sequence'])):
    coeff = checkRecurrence(seq,2)
    if coeff:
        predict = predictNextTerm(seq, coeff)
        order2Seq += 1
        data['Solved'][i] = 'rr2'
        
print(f'Solved: {order2Seq} ({round((order2Seq*100)/len(data),2)}%)')


Solved: 3269 (1.44%)


### 3rd Order Sequences

In [12]:
order3Seq = 0
for i, seq in tqdm(enumerate(data['Sequence'])):
    coeff = checkRecurrence(seq,3)
    if coeff:
        predict = predictNextTerm(seq, coeff)
        order3Seq += 1
        data['Solved'][i] = 'rr3'
        
print(f'Solved: {order3Seq} ({round((order3Seq*100)/len(data),2)}%)')


Solved: 3843 (1.69%)


### 4th Order Sequences

In [13]:
order4Seq = 0
for i, seq in tqdm(enumerate(data['Sequence'])):
    coeff = checkRecurrence(seq,4)
    if coeff:
        predict = predictNextTerm(seq, coeff)
        order4Seq += 1
        data['Solved'][i] = 'rr4'
        
print(f'Solved: {order4Seq} ({round((order4Seq*100)/len(data),2)}%)')


Solved: 3307 (1.45%)


In [14]:
data['Solved'].value_counts()

       217309
rr3      3832
rr4      3307
rr2      3242
Name: Solved, dtype: int64